# Part 4. 결측치(Missing Value) 처리의 논리

이 장은 전처리에서 반드시 마주치게 되는 결측치를 어떻게 이해하고 다룰지를 정리한다. <br>
결측치는 오류가 아니라, 현실 데이터가 가진 상태 중 하나다.

## 1) 결측치란 무엇인가?
결측치란 **값이 비어 있거나, 알 수 없는 상태**를 의미한다.
- 숫자형 결측 → NaN
- 날짜형 결측 → NaT
- 문자열/기타 → NA 또는 NaN

가 대표적인 예시이다

기억하자
- 결측치는 **이상한 데이터가 아닌 현실 데이터**를 그대로 반영한 상태다.

## 2) 결측치가 생기는 이유
결측치는 실수로만 생기지 않는다. 대부분 데이터가 만들어지는 과정의 결과다.

1. 입력의 누락
    - 사람이 값을 입력하지 않음
2. 수집 오류
    - 시스템, 센서, 로그 문제
3. 변환 실패
    - dtype 변환 중 해석 불가 값
    - errors="coerce" 사용 결과
4. 결합(merge) 실패
    - 다른 테이블과 매칭되지 않음
5. 원래 존재하지 않는 값
    - 옵션 미선택, 해당 항목 없음

결측치는 실수라기보다 데이터 생성 과정의 수많은 결과중 하나다.<br>
"왜 없을까?"를 생각해야 하는 대상이지 무조건 제거해야 할 대상은 아니다.

## 3) 결측치 처리의 기본 흐름

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "date": ["2026-01-01", "2026/01/02", "2026-01-03"],
    "menu": ["Americano", "Latte", "Mocha"],
    "price": ["4500원", "5,000", None],
    "qty": ["2", 1, None],
    "paid": ["TRUE", "FALSE", True]
})

df["date"] = pd.to_datetime(df["date"], errors="coerce") # datetime으로 형변환
df["qty"] = pd.to_numeric(df["qty"], errors="coerce") # 숫자형으로 형변환
# price 문자열 정리
print("price 컬럼 문자열 정리")
df["price"] = (
    df["price"]
    .astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

print("바뀐 데이터프레임의 정보")
df.info()

## 4) 결측치의 확인
결측치 확인은 전처리의 기본 점검 단계다.
- 어디에 결측치가 있는지
- 얼마나 있는지

이 단계에서는 아직 처리하지 않고, 현상만 정확히 파악하는 단계다.

대표적인 결측치 확인 함수
- isna(): 각 값이 결측치(NaN, NaT, None)인지 여부를 True / False로 반환하는 함수다.

In [ ]:
print("date 컬럼 결측치 확인")
print(df["date"].isna())

print("\nprice 컬럼 결측치 확인")
print(df["price"].isna())

print("\nqty 컬럼 결측치 확인")
print(df["qty"].isna())

In [ ]:
# tip notna()는 isna의 반대. 결측값이 아닌 값을 표현한다.
print("date 컬럼 결측치 확인(반전)")
print(df["date"].notna())

# tip isna().sum()을 통해 결측값의 갯수를 샐수도 있다.
print("\n컬럼별 결측치 확인")
print(df.isna().sum())

In [ ]:
# .any()를 사용해 결측치 존재여부만 빠르게 확인 가능하다.
print("date 컬럼에 결측치가 있는가?")
print(df["date"].isna().any())

print("\nprice 컬럼에 결측치가 있는가?")
print(df["price"].isna().any())

print("\nqty 컬럼 결측치 확인")
print(df["qty"].isna().any())

print("\nmenu 컬럼 결측치 확인")
print(df["menu"].isna().any())

## 5) 결측치 처리 전략 3가지
결측치 처리에는 대표적으로 세 가지 선택지가 있다. <br>
중요한 것은 **정답은 없고, 목적만 있다**는 점이다.

### 1. 삭제 (drop)
```python
df_drop = df.dropna()

```
결측치가 포함된 행이나 컬럼을 아예 제거하는 방식이다.<br>
다음과 같은 상황에 사용하긴 하나 되도록 자제하도록 하자
- 결측치 비율이 매우 낮고
- 해당 데이터가 분석에 중요하지 않을 때

사용한다.

특징
- 장점: 단순, 해석이 깔끔
- 단점: 데이터가 줄어들어 분석력이 떨어질 수 있음

가장 쉬운 방법이지만, 무조건적인 삭제는 정보 손실로 이어질 수 있기 때문에 신중하게 사용해야 한다.

In [ ]:
print("결측치가 있는 행 삭제")
df_drop = df2.dropna()

print(df_drop)

### 2. 대체 (fill)
```python
df_fill = df.copy()

```
결측치를 의미 있는 값으로 채우는 방식이다.
- 데이터 양을 유지하고 싶을 때
- "비어 있음"이 어느 정도 의미를 가질 때

<br>

채워 넣는 값은 다양하지만, 대표적으로 다음과 같다.
- 평균, 중앙값, 최빈값
- 0, False, “Unknown”

하지만 항상 질문해야 한다.
> 이 값이 0이라는 게 의미적으로 맞는가?

의미 없이 채운 값은 분석 결과를 왜곡한다.

In [ ]:
df_fill = df.copy()

print("price 결측치를 0으로 대체")
df_fill["price"] = df_fill["price"].fillna(0)
print(df_fill)

print("\ndate 결측치를 특정 날짜로 대체")
df_fill["date"] = df_fill["date"].fillna(pd.Timestamp("2026-01-01"))
print(df_fill)

In [ ]:
print("컬럼별 결측치 확인")
print(df_fill.isna().sum())

결측값을 fillna로 다른값으로 채운 price, date는 0인것을 확인할수 있다. 

다시한번 기억해야할것은 결측값은 신중하게 다뤄야 한다.<br>
자칫 잘못 다루면 오히려 결측값이 있을때 보다 못할수도 있다.

### 3. 추정 (개념만 이해)
다른 값들을 이용해 결측치를 추정하는 방식이다.
- 모델 기반 추정
- 주변 값 기반 추정

초심자 단계에서는 개념만 이해하고 넘어간다. <br>
실무에서도 항상 필요한 방법은 아니다.